# **The Battle Of Neighbourhoods Assignment (Week5)**
## **Part 1 (Introduction)**
A description of the problem and a discussion of the background. (15 marks)
Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

### Business Problem:
In this assignment we talk about How good Rio de Janeiro is to open a restaurant. And would it be a place with lots of reviews from customers? Which place seems to be the most popular and the last question would be Is there a more popular restaurant in the area and what cuisines are preferred around?

The problem will be solved using the Foursquare API, looking at User Count and Tip Count. This issue will help everyone who wants to open a restaurant in Rio de Janeiro and wants to know if it is a good option where they intend to open.

---


## **Part 2**
A description of the data and how it will be used to solve the problem. (15 marks)
Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

This submission will eventually become your Data section in your final report. So I recommend that you push the report (having your Data section) to your Github repository and submit a link to it.

###About Data:
The data that will are using is Foursquare's data for Rio de Janeiro city. We will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant.

---



#Importing Important Libraries needed for Assignment

In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


# Fetching Location of Rio de Janeiro

In [19]:
# Get longitude and latitude for Toronto
address = 'Rio de Janeiro, Rio de Janeiro'

geolocator = Nominatim(user_agent="rio_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rio de Janeiro is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rio de Janeiro is -22.9110137, -43.2093727.


# Setting up FourSquare

In [20]:
# Set up Foursquare
CLIENT_ID = '4UNRI4GOCXF002WSCBY0OXWPSBUP1VABD0QKXABMSVGR34IC' # your Foursquare ID
CLIENT_SECRET = 'TOEMP4WMP4Q03AT2GBPAZF5KC13U5LB5BEZUBOZRA4LLP3QC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4UNRI4GOCXF002WSCBY0OXWPSBUP1VABD0QKXABMSVGR34IC
CLIENT_SECRET:TOEMP4WMP4Q03AT2GBPAZF5KC13U5LB5BEZUBOZRA4LLP3QC


In [21]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [22]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4UNRI4GOCXF002WSCBY0OXWPSBUP1VABD0QKXABMSVGR34IC&client_secret=TOEMP4WMP4Q03AT2GBPAZF5KC13U5LB5BEZUBOZRA4LLP3QC&ll=-22.9110137,-43.2093727&v=20180605&query=Restaurant&radius=50000&limit=100'

In [0]:
results = requests.get(url).json()

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.crossStreet
0,56e04e1d38fa3ae11155b863,Riá Restaurant,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",v-1588540267,False,"Rua Riachuelo, 242",-22.914733,-43.188829,"[{'label': 'display', 'lat': -22.914733, 'lng'...",2146,BR,Rio de Janeiro,RJ,Brasil,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]",NaN,NaN
1,4ece6782cc219860f521dd5d,Restaurante Pitada,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",v-1588540267,False,NaN,-22.909734,-43.191598,"[{'label': 'display', 'lat': -22.9097338088607...",1828,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, Brasil]",NaN,NaN
2,50e0b634e4b0f94bd8a38f47,Restaurant El Duero,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1588540267,False,Navio Soberano,-22.892851,-43.189795,"[{'label': 'display', 'lat': -22.8928508209919...",2849,BR,Rio de Janeiro,RJ,Brasil,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]",NaN,NaN
3,4ba85046f964a5201fd639e3,Restaurante Rampinha,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",v-1588540267,False,"Pç. da Bandeira, 201",-22.911713,-43.214886,"[{'label': 'display', 'lat': -22.9117127074659...",570,BR,Rio de Janeiro,RJ,Brasil,"[Pç. da Bandeira, 201, Rio de Janeiro, RJ, 202...",20270-150,NaN
4,54fc90bc498e67112301efff,Restaurante Drink's,"[{'id': '52e81612bcbc57f1066b7a00', 'name': 'C...",v-1588540267,False,Travessa Angustura,-22.913848,-43.213167,"[{'label': 'display', 'lat': -22.9138483642793...",500,BR,Rio de Janeiro,RJ,Brasil,"[Travessa Angustura (Rua do Matoso), Rio de Ja...",NaN,Rua do Matoso


# Cleaning and Keeping Required Data

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,crossStreet,id
0,Riá Restaurant,Bistro,"Rua Riachuelo, 242",-22.914733,-43.188829,"[{'label': 'display', 'lat': -22.914733, 'lng'...",2146,BR,Rio de Janeiro,RJ,Brasil,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]",NaN,NaN,56e04e1d38fa3ae11155b863
1,Restaurante Pitada,Brazilian Restaurant,NaN,-22.909734,-43.191598,"[{'label': 'display', 'lat': -22.9097338088607...",1828,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, Brasil]",NaN,NaN,4ece6782cc219860f521dd5d
2,Restaurant El Duero,Restaurant,Navio Soberano,-22.892851,-43.189795,"[{'label': 'display', 'lat': -22.8928508209919...",2849,BR,Rio de Janeiro,RJ,Brasil,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]",NaN,NaN,50e0b634e4b0f94bd8a38f47
3,Restaurante Rampinha,Brazilian Restaurant,"Pç. da Bandeira, 201",-22.911713,-43.214886,"[{'label': 'display', 'lat': -22.9117127074659...",570,BR,Rio de Janeiro,RJ,Brasil,"[Pç. da Bandeira, 201, Rio de Janeiro, RJ, 202...",20270-150,NaN,4ba85046f964a5201fd639e3
4,Restaurante Drink's,Comfort Food Restaurant,Travessa Angustura,-22.913848,-43.213167,"[{'label': 'display', 'lat': -22.9138483642793...",500,BR,Rio de Janeiro,RJ,Brasil,"[Travessa Angustura (Rua do Matoso), Rio de Ja...",NaN,Rua do Matoso,54fc90bc498e67112301efff


# Creating Visualization

In [26]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map